In [91]:
import pandas as pd
import hopsworks
import os
import requests

In [92]:
player_stats_gw_url = "https://fantasy.premierleague.com/api/event/{gw}/live/"
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [93]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")
    
project = hopsworks.login()
print(project.description)

2025-01-05 11:58:51,363 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 11:58:51,369 INFO: Initializing external client
2025-01-05 11:58:51,369 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 11:58:52,567 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1159321
Default project


In [94]:
general_info = requests.get(bootstrap_url).json()

In [95]:
with open ('./state/prev_gameweek.txt', 'r') as file:
    prev_gameweek = file.readline()
print(prev_gameweek)

19


In [96]:
prev_gameweek = next(
    (gw for gw in general_info['events'] if gw['id'] == int(prev_gameweek)), None
)
prev_finished = prev_gameweek['finished']
prev_finished

True

In [97]:
current_gameweek = next(
    (gw for gw in general_info['events'] if gw['is_current']), None
)

In [98]:
lagged_features = ['minutes', 'goals_scored', 'assists', 'clean_sheets',
                   'goals_conceded', 'own_goals', 'penalties_saved',
                   'penalties_missed', 'yellow_cards', 'red_cards', 'saves',
                   'bonus', 'bps', 'influence', 'creativity', 'threat',
                   'ict_index', 'starts', 'expected_goals', 'expected_assists',
                   'expected_goal_involvements', 'expected_goals_conceded',
                   'in_dreamteam']


In [99]:
if prev_gameweek != current_gameweek["id"] and prev_finished:


    prev_stats = player_stats_gw_url.format(gw=prev_gameweek["id"])
    prev_stats = requests.get(prev_stats).json()
    prev_stats_df = pd.DataFrame(prev_stats["elements"]).drop(columns=["explain", "modified"])
    prev_stats_df = pd.concat(
        [prev_stats_df.drop(columns=["stats"]), prev_stats_df["stats"].apply(pd.Series)], 
        axis=1
    )
    prev_stats_df["gameweek"] = current_gameweek["id"]

    prev_stats_df.rename(columns={col: f"prev_{col}" for col in prev_stats_df.columns if col not in ["id", "gameweek", "total_points"]}, inplace=True)

    prev_stats_df = prev_stats_df.dropna(subset=[f'prev_{col}' for col in lagged_features] + ['total_points'])

    prev_stats_df = prev_stats_df.astype({col: 'float64' for col in prev_stats_df.columns if col not in ["id", "gameweek", "total_points"]})

    fs = project.get_feature_store() 

    player_fg = fs.get_feature_group(
        name="player_features",
        version=1,
    )
    
    player_fg.insert(prev_stats_df)

    with open ('./state/prev_gameweek.txt', 'w') as file:
        file.write(str(current_gameweek["id"]))

    print("Inserted data for gameweek", current_gameweek["id"])

Uploading Dataframe: 100.00% |██████████| Rows 709/709 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: player_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1159321/jobs/named/player_features_1_offline_fg_materialization/executions
Inserted data for gameweek 20
